In [ ]:
import sys
sys.path.append(r'c:\ai_project01\Pytorch_Retinaface')

In [ ]:
import os
import torch
import cv2
import numpy as np
from models.retinaface import RetinaFace
from data import cfg_re50
from layers.functions.prior_box import PriorBox
from utils.nms.py_cpu_nms import py_cpu_nms
from utils.box_utils import decode
torch.cuda.is_available()

In [ ]:
device = "cuda"
cfg = cfg_re50
net = RetinaFace(cfg=cfg, phase='test').to(device)
pretrained_path=r'C:\ai_project01\Pytorch_Retinaface\weights\Resnet50_Final.pth'
state_dict = torch.load(pretrained_path, map_location=device, weights_only=True)
new_state_dict={}
for k,v in state_dict.items():
    if k.startswith("module."):
        new_state_dict[k[7:]] = v
    else:
        new_state_dict[k] = v
net.load_state_dict(new_state_dict, strict = True)

In [ ]:
net.eval()
image_dirs = {
    'mask_on' : r'C:\ai_project01\mask_images\mask_on',
    'no_mask' : r'C:\ai_project01\mask_images\no_mask'
}

label_dir = r'C:\ai_project01\labels'
os.makedirs(label_dir, exist_ok=True)
class_ids = {
    'mask_on': 0,
    'no_mask': 1
}

for mask_status, image_dir in image_dirs.items():
    for img_file in os.listdir(image_dir):
        if img_file.lower().endswith(('png', 'jpg', 'jpeg')):
            image_path = os.path.join(image_dir, img_file)
            img_raw = cv2.imread(image_path, cv2.IMREAD_COLOR)
            img = np.float32(img_raw)
            im_height, im_width, _ = img.shape
            scale = torch.Tensor([im_width, im_height, im_width, im_height]).to(device)
            img -= (104,117,123)
            img = img.transpose(2,0,1)
            img = torch.from_numpy(img).unsqueeze(0).to(device)
            loc, conf, landms = net(img)
            priorbox = PriorBox(cfg, image_size = (im_height, im_width))
            priors = priorbox.forward().to(device)
            prior_data = priors.data
            boxes = decode(loc.data.squeeze(0), prior_data, cfg['variance'])
            boxes = boxes * scale
            scores = conf.squeeze(0).data.cpu().numpy()[:,1]
            confidence_threshold = 0.5
            top_indices = np.where(scores > confidence_threshold)[0]
            boxes = boxes[top_indices]
            scores = scores[top_indices]
            dets = np.hstack((boxes.cpu().numpy(), scores[:, np.newaxis])).astype(np.float32, copy=False)
            keep = py_cpu_nms(dets, 0.3)
            dets = dets[keep, :]
            yolo_labels = []
            for b in dets:
                x1, y1, x2, y2 = b[:4]
                x_center = ((x1 + x2) / 2) / im_width
                y_center = ((y1 + y2) / 2) / im_height
                bbox_width = (x2 - x1) / im_width
                bbox_height = (y2 - y1) / im_height
                class_id = class_ids[mask_status]
                yolo_labels.append(f"{class_id} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}")
            label_path = os.path.join(label_dir, os.path.splitext(img_file)[0] + '.txt')
            with open(label_path, 'w') as f:
                f.write('\n'.join(yolo_labels))
            print(f"Processed: {img_file} -> {label_path}")
            
            